In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [42]:
fa = pd.read_excel("../Data/Food Insecurity Rates -Tracts - Illinois.xlsx", sheet_name='Full_counties')
fa.head()

,Tract,FIPS,State,Total Population,2018 Food Insecurity,2018 Food Insecurity #,[C] 2020 Food Insecurity,[C] 2020 Food Insecurity #,[C] 2020 Food Insecurity # Change,[C] Adjusted Annual Unemployment Change
0,"Census Tract 1001, Cook County, Illinois",17031,IL,"5,563",0.099,550,0.149,820,270,0.07
1,"Census Tract 1002, Cook County, Illinois",17031,IL,"6,872",0.060,410,0.110,750,340,0.07
2,"Census Tract 1003, Cook County, Illinois",17031,IL,"6,186",0.074,460,0.123,770,310,0.07
3,"Census Tract 1004, Cook County, Illinois",17031,IL,"3,364",0.055,180,0.104,350,170,0.07
4,"Census Tract 1005, Cook County, Illinois",17031,IL,"5,696",0.052,290,0.101,570,280,0.07


In [41]:
basepath = "../Data/PolicyMap Data/"
filename = "average_household_size.csv"
first = pd.read_csv(basepath + filename)
#first need to remove some documentation text that is added by policymaps to census tract column
first = first[first['Census Tract'].apply(lambda s: 'IL' in s)]
first.head()

,Census Tract,FIPS Code,Formatted FIPS,"Estimated average size of a household, between 2014-2018."
0,"17031990000, IL",1.703199e+10,"=""17031990000""",NaN
1,"17031804201, IL",1.703180e+10,"=""17031804201""",3.08
2,"17031823801, IL",1.703182e+10,"=""17031823801""",2.45
3,"17031824004, IL",1.703182e+10,"=""17031824004""",2.56
4,"17031820202, IL",1.703182e+10,"=""17031820202""",2.64


In [87]:
def preprocess_census_tract_pm(s):
    """
    Preprocesses a census tract string (s) 
    and returns only the most local identifier (removes the state, and county sub-codes)
    """
    #remove all characters, commas, and whitespace
    s = s.replace("IL", "").replace(",", "").strip()
    #get the last six characters (digits) and return as int
    return int(s[-6:])

,Census Tract,FIPS Code,Formatted FIPS,"Estimated average size of a household, between 2014-2018.",local_census_tract
0,"17031990000, IL",1.703199e+10,"=""17031990000""",NaN,990000
1,"17031804201, IL",1.703180e+10,"=""17031804201""",3.08,804201
2,"17031823801, IL",1.703182e+10,"=""17031823801""",2.45,823801
3,"17031824004, IL",1.703182e+10,"=""17031824004""",2.56,824004
4,"17031820202, IL",1.703182e+10,"=""17031820202""",2.64,820202


In [112]:
def preprocess_census_tract_fa(s):
    regex_pattern = r'[0-9]+.?[0-9]+'
    census_tract_code = re.findall(regex_pattern, s)[0]
    return int(float(census_tract_code) * 100)


In [102]:
with open("../Data/PolicyMap Data/file_list.txt", 'r') as f:
    filenames = list(map(lambda s: s.replace("\n", ""), f.readlines()))
    filenames = list(filter(lambda fn: fn != 'filelist.txt', filenames))
filenames

['average_household_size.csv',
 'avg_travel_time_to_work.csv',
 'household_computer_access.csv',
 'life_expectancy_2010_2015.csv',
 'median_age.csv',
 'median_home_loan_2018.csv',
 'median_house_value.csv',
 'median_household_income.csv',
 'median_leverage_ratio_2018.csv',
 'non_federal_jobs_2017.csv',
 'number_housing_units.csv',
 'percentage_bachelor_degree_2014_2018.csv',
 'percentage_disability.csv',
 'percentage_hispanic.csv',
 'percentage_male.csv',
 'percentage_non_white.csv',
 'percentage_nonenglish.csv',
 'percentage_poverty.csv',
 'public_school_enrollment_k_12.csv',
 'social_vulnerability_index_2018.csv',
 'thiel_index_segregation.csv']

In [103]:

#write a for loop to automate the joining process
#initialize merged_data with the feeding america dataset
merged_data = fa.copy()
merged_data['local_census_tract'] = merged_data['Tract'].apply(preprocess_census_tract_fa)

for filename in filenames:
    print("Working on:", filename)
    #read the policymap data in
    pm = pd.read_csv(basepath + filename)
    #remove bogus rows that are part of policymaps data
    pm = pm[pm['Census Tract'].apply(lambda s: 'IL' in s)]
    #preprocess the policymap census tract code
    pm['local_census_tract'] = pm['Census Tract'].apply(preprocess_census_tract_pm)
    #drop extraneous columns in policymap data before merge
    pm.drop(['Census Tract', 'FIPS Code', 'Formatted FIPS'], axis = 1, inplace = True)
    merged_data = pd.merge(merged_data, pm, on = 'local_census_tract', how = 'left')
    
    


Working on: average_household_size.csv
Working on: avg_travel_time_to_work.csv
Working on: household_computer_access.csv
Working on: life_expectancy_2010_2015.csv
Working on: median_age.csv
Working on: median_home_loan_2018.csv
Working on: median_house_value.csv
Working on: median_household_income.csv
Working on: median_leverage_ratio_2018.csv
Working on: non_federal_jobs_2017.csv
Working on: number_housing_units.csv
Working on: percentage_bachelor_degree_2014_2018.csv
Working on: percentage_disability.csv
Working on: percentage_hispanic.csv
Working on: percentage_male.csv
Working on: percentage_non_white.csv
Working on: percentage_nonenglish.csv
Working on: percentage_poverty.csv
Working on: public_school_enrollment_k_12.csv
Working on: social_vulnerability_index_2018.csv
Working on: thiel_index_segregation.csv


In [111]:
dest_path = "../Data/fa_pm_merged_data.csv"
merged_data.to_csv(dest_path)

In [4]:
fapm = pd.read_csv("../Data/FA_PM/fa_pm_merged_data.csv")
fapm.head()

,Tract,fips,state,total_population,food_insecurity_rate_2018,food_insecurity_num_2018,food_insecurity_rate_2020,food_insecurity_num_2020,food_insecurity_num_change_2020,unemployment_change,...,proportion_bachelors_degree,proportion_disabled,proportion_hispanic,prop_men,prop_nonwhite,prop_nonenglish_speaking,prop_families_poverty,prop_students_in_public_school,social_vulnerability_index,thiel_racial_segregation_index
0,"Census Tract 1001, Cook County, Illinois",17031,IL,"5,563",0.099,550,0.149,820,270,0.07,...,26.32,10.84,16.07,48.30,4.31,12.54,6.60,53.92,0.36,0.24
1,"Census Tract 1002, Cook County, Illinois",17031,IL,"6,872",0.060,410,0.110,750,340,0.07,...,22.21,6.48,11.04,47.22,10.84,5.79,11.35,79.56,0.34,0.20
2,"Census Tract 1003, Cook County, Illinois",17031,IL,"6,186",0.074,460,0.123,770,310,0.07,...,25.82,17.10,10.17,44.94,11.75,7.64,2.00,68.25,0.30,0.22
3,"Census Tract 1004, Cook County, Illinois",17031,IL,"3,364",0.055,180,0.104,350,170,0.07,...,28.45,8.98,13.97,51.25,7.52,3.97,0.93,68.16,0.27,0.15
4,"Census Tract 1005, Cook County, Illinois",17031,IL,"5,696",0.052,290,0.101,570,280,0.07,...,25.93,12.90,10.45,47.82,12.34,14.19,0.00,68.67,0.28,0.20


In [30]:
evanston_tracts = [808900, 808800, 808701, 809000, 809100, 809300, 809200, 808702, 809600, 
                   809500, 809400, 809700, 809800, 809900, 810301, 
                  810302, 810100, 810200, 810000]
fapm_ev = fapm[fapm['local_census_tract'].isin(evanston_tracts)]
fapm_ev.head()

,Tract,fips,state,total_population,food_insecurity_rate_2018,food_insecurity_num_2018,food_insecurity_rate_2020,food_insecurity_num_2020,food_insecurity_num_change_2020,unemployment_change,...,proportion_bachelors_degree,proportion_disabled,proportion_hispanic,prop_men,prop_nonwhite,prop_nonenglish_speaking,prop_families_poverty,prop_students_in_public_school,social_vulnerability_index,thiel_racial_segregation_index
873,"Census Tract 8087.02, Cook County, Illinois",17031,IL,"5,255",0.117,620,0.167,880,260,0.07,...,34.86,5.90,9.36,45.40,34.33,5.38,0.00,92.96,0.40,0.04
874,"Census Tract 8088, Cook County, Illinois",17031,IL,"3,732",0.049,180,0.099,360,180,0.07,...,35.40,5.18,3.94,47.91,13.99,5.76,2.87,71.92,0.16,0.25
875,"Census Tract 8089, Cook County, Illinois",17031,IL,"4,175",0.036,150,0.086,360,210,0.07,...,32.18,6.88,4.26,48.12,7.19,3.61,2.24,88.43,0.19,0.25
876,"Census Tract 8090, Cook County, Illinois",17031,IL,"4,153",0.045,190,0.095,400,210,0.07,...,34.32,9.11,1.69,45.94,14.54,3.30,0.00,83.72,0.29,0.29
877,"Census Tract 8091, Cook County, Illinois",17031,IL,"3,293",0.048,160,0.097,320,160,0.07,...,29.65,9.95,4.28,43.40,15.70,3.17,2.33,91.34,0.34,0.24


In [31]:
#808701 is not showing up in the fapm[local_census_tract] dataset
set(evanston_tracts)- set(fapm_ev['local_census_tract'].to_numpy())

{808701}

In [34]:
#write the data to data folder
fapm_ev.to_csv('../Data/FA_PM/fa_pm_evanston.csv')